<a href="https://colab.research.google.com/github/SarveshPatil99/Adversarial-Robustness-Enhancement/blob/main/TML_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Change the current working directory
new_directory = "/content/drive/MyDrive/TML_Project/data/"
os.chdir(new_directory)

In [1]:

select_a_dataset = "StyleGAN3 10000" # @param ["StyleGAN3 5000", "StyleGAN3 10000", "StyleGAN2 5000"]

if select_a_dataset == "StyleGAN3 5000":
  !gdown https://drive.google.com/uc?id=1-BTtMGzH8925FtIu20KbkfmmSz3LNTer
  !unzip -q original_dataset.zip
elif select_a_dataset == "StyleGAN3 10000":
  !gdown https://drive.google.com/uc?id=18MZZwgKjTGlB2Y4Esw_lRIvbpOaANbnh
  !unzip -q original_dataset_stylegan3_10000.zip
elif select_a_dataset == "StyleGAN2 5000":
  !gdown https://drive.google.com/uc?id=1ZBxnHMArN2-HAdViUFKarKxuXVk4sREk
  !unzip -q original_dataset_stylegan2_5000.zip

Downloading...
From: https://drive.google.com/uc?id=18MZZwgKjTGlB2Y4Esw_lRIvbpOaANbnh
To: /content/original_dataset_stylegan3_10000.zip
100% 2.12G/2.12G [00:23<00:00, 91.9MB/s]


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# Define the device to train on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transforms for the training, validation, and testing sets
transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

# Load the datasets with ImageFolder
data_dir = 'original'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transforms[x])
                  for x in ['train', 'val', 'test']}

# Define a dataloader
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val', 'test']}

# Define the model
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the GPU
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# A function to train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return model

# Train the model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can als

Epoch 0/24
----------
train Loss: 0.6068 Acc: 0.6594
val Loss: 0.6330 Acc: 0.6910
Epoch 1/24
----------
train Loss: 0.2888 Acc: 0.8772
val Loss: 0.4052 Acc: 0.8250
Epoch 2/24
----------
train Loss: 0.1271 Acc: 0.9514
val Loss: 0.3702 Acc: 0.8635
Epoch 3/24
----------
train Loss: 0.0722 Acc: 0.9723
val Loss: 0.3378 Acc: 0.8820
Epoch 4/24
----------
train Loss: 0.0582 Acc: 0.9784
val Loss: 0.4662 Acc: 0.8545
Epoch 5/24
----------
train Loss: 0.0465 Acc: 0.9817
val Loss: 0.3296 Acc: 0.8970
Epoch 6/24
----------
train Loss: 0.0349 Acc: 0.9881
val Loss: 0.3059 Acc: 0.9095
Epoch 7/24
----------
train Loss: 0.0151 Acc: 0.9956
val Loss: 0.1842 Acc: 0.9350
Epoch 8/24
----------
train Loss: 0.0078 Acc: 0.9980
val Loss: 0.1816 Acc: 0.9345
Epoch 9/24
----------
train Loss: 0.0057 Acc: 0.9986
val Loss: 0.1825 Acc: 0.9365
Epoch 10/24
----------
train Loss: 0.0060 Acc: 0.9984
val Loss: 0.1796 Acc: 0.9370
Epoch 11/24
----------
train Loss: 0.0053 Acc: 0.9991
val Loss: 0.1758 Acc: 0.9395
Epoch 12/24
--

In [3]:
# Function to test the model accuracy on the test set
def test_model(model, dataloaders, device):
    model.eval()  # Set the model to evaluation mode
    running_corrects = 0

    # No need to track gradients for testing
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Statistics
            running_corrects += torch.sum(preds == labels.data)

    # Calculate accuracy
    epoch_acc = running_corrects.double() / len(image_datasets['test'])
    print(f'Test Acc: {epoch_acc:.4f}')

# Test the model
test_model(model, dataloaders, device)

Test Acc: 0.9260


In [4]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'resnet34_model.pth')

# Function to load the model
def load_model(model_path):
    # Make sure to provide the same number of classes as during training for the fully connected layer
    model = models.resnet34(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)  # Assuming we have 2 classes

    # Load the saved state dictionary into the model
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    return model

# Load the model
model = load_model('resnet34_model.pth')

# Function to evaluate the model
def evaluate_model(model, dataloaders, phases=['train', 'val', 'test']):
    model.eval()  # Set the model to evaluation mode
    results = {}

    with torch.no_grad():
        for phase in phases:
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                # Statistics
                running_corrects += torch.sum(preds == labels.data)

            # Calculate accuracy
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            results[phase] = epoch_acc.item()
            print(f'{phase} Acc: {epoch_acc:.4f}')

    return results

# Evaluate the model
results = evaluate_model(model, dataloaders)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


train Acc: 1.0000
val Acc: 0.9360
test Acc: 0.9260


In [5]:
!pip install tensorflow classification-models

  Preparing metadata (setup.py) ... done
  Created wheel for classification-models: filename=classification_models-0.1-py3-none-any.whl size=4726 sha256=15e4efd1e26006091c85a5bcef33121b554033200970f4dd3cc9cd59c74fcd4d
  Stored in directory: /root/.cache/pip/wheels/63/f6/19/2e61ad3997efefe7079f3005074d68e103180974b16cb3f870
Successfully built classification-models
